In [3]:
# Шаардлагатай сангуудыг суулгаж буй хэсэг
import numpy as np
import array
from keras.datasets import mnist
import keras.utils
from sklearn.metrics import accuracy_score
import math

# x = float('nan')
# math.isnan(x)

Using TensorFlow backend.


In [4]:
# MNIST DATASET буюу 60000 ширхэг сургалтын 0-9 хооронд бичсэн гар бичмэлийн өгөгдөл болон зөв хариу нь өгөдсөн ба
# шалгалтын өгөгдөл нь 10000 ширхэг бөгөөд мөн зөв хариу нь өгөгдсөн.

#keras -ын сан ашиглан өгөгдлөө татаж авч буй хэсэг
(train_data,train_label),(test_data,test_label) = mnist.load_data()

# зөв хариуг нь encoding хийж буй хэсэг

train_label_data_ = keras.utils.to_categorical(train_label, 10)
test_label_data_ = keras.utils.to_categorical(test_label, 10)

# Хар цагаан зураг нь 0-255 хооронд байдаг учир үүнийг 0-1 хооронд болгохын
# тулд pixel бүрийг нь 255 хувааж буй хэсэг

train_data = train_data / 255
test_data = test_data / 255

# Энд сургалтын өгөгдөл болон шалгалтын өгөгдлийн хэмжээг (60000,748) болгож 
# байна. Уг өгөгдөл нь эхлээд (60000,28,28) гэсэн хэмжээтэй орж ирж байгаа.

train_data_data_ = train_data.reshape(60000,784)
test_data_data_ = test_data.reshape(10000,784)

# 1. mean = 0, var = 1 baihaar data normalize hiij baigaa heseg

train_label_data = (train_label_data_ - np.mean(train_label_data_))/np.std(train_label_data_)
test_label_data = (test_label_data_ - np.mean(test_label_data_))/np.std(test_label_data_)
train_data_data = (train_data_data_ - np.mean(train_data_data_))/np.std(train_data_data_)
test_data_data = (test_data_data_ - np.mean(test_data_data_))/np.std(test_data_data_)

print('train_label_data_ mean:',np.mean(train_label_data))
print('train_label_data_ var:',np.var(train_label_data))
print('train_label_data_ std:',np.std(train_label_data))

train_label_data_ mean: 6.037553e-09
train_label_data_ var: 0.9999998
train_label_data_ std: 0.9999999


In [5]:
print("train_label shape:",train_label.shape)

train_label shape: (60000,)


In [6]:
class NeuralNetwork:
    
    def __init__(self,epoch_size,batch_size,input_size,hidden_size,output_size,learn_rate):
        
        np.random.seed(0)
        
        self.epoch = epoch_size
        
        self.batch = batch_size
        
        self.input = input_size
        
        self.hidden = hidden_size
        
        self.output = output_size
        
        self.lr = learn_rate
        
        self.w1 = np.random.normal(loc = 0,scale = 0.1,size=(self.input, self.hidden))
        
        self.b1 = np.zeros((1,self.hidden))
        
        self.w2 = np.random.normal(loc = 0,scale = 0.1,size=(self.hidden, self.output))
        
        self.b2 = np.zeros((1,self.output))
        
    def relu(self,x):
        return np.maximum(0,x)
    
    def step(self,x):
        return np.heaviside(x,1)
    
    def softmax(self,k):
        
        k = k.T
        
        e_x = np.exp(k - np.max(k))
        
        soft = e_x / e_x.sum(axis=0)
        
        return soft.T
    
    def forward_propagation(self,input_data):
        
        h = np.dot(input_data,self.w1) + self.b1
        
        H = self.relu(h)
        
        z = np.dot(H,self.w2) + self.b2
        
        y = self.softmax(z)

        return y,z,H,h
    
    def error_formula(self,y,y_hat):
        loss = []
        for i in range(self.batch):
#             print("example:",-np.log(y[i][y_hat[i]]))
#             print("y shape:",y.shape)
#             print("y value:",y)
#             print("y_hat shape:",y_hat.shape)
#             print("y_hat value:",y_hat)
            correct_logprobs = -np.log(y[i,y_hat[i]])
            loss.append(correct_logprobs)
        return np.mean(loss)
    
    def back_propagation(self,input_data,y_hat,y1_hat):
        
        y,z,H,h = self.forward_propagation(input_data)
        
        error = (y - y_hat)
        
        loss = self.error_formula(y,y1_hat)
        
        d_w1 = np.dot(input_data.T,self.step(h)*np.dot(self.w2,error.T).T) # (784:16)
        
        d_b1 = self.step(h)*np.dot(self.w2,error.T).T # (batch:16)
        
        d_w2 = np.dot(H.T,error) # (16:10)
        
        d_b2 = error # (batch:10)
        
        if loss > 0:
            self.w1 = self.w1 - d_w1*self.lr
            self.b1 = self.b1 - d_b1*self.lr
            self.w2 = self.w2 - d_w2*self.lr 
            self.b2 = self.b2 - d_b2*self.lr
        else:
            self.w1 = self.w1 + d_w1*self.lr
            self.b1 = self.b1 + d_b1*self.lr
            self.w2 = self.w2 + d_w2*self.lr 
            self.b2 = self.b2 + d_b2*self.lr
        
        return loss,y,self.w1,self.w2,self.b1,self.b2
    
    def predict(self,test_data,test_label,w1,w2,b1,b2):
        
        h = np.dot(test_data,w1) + b1
        
        H = self.relu(h)
        
        z = np.dot(H,w2) + b2
        
        y = self.softmax(z)
        
        loss = self.error_formula(y,test_label)
        
        return loss

In [7]:
# print()
train_label.shape

(60000,)

In [9]:
input_size = 784
hidden_size = 128
epoch_size = 5
batch_size = 50
output_size = 10
learn_rate = 0.000001

print("train data shape:",train_data_data.shape)

super_w1 = np.zeros((input_size,hidden_size))
super_w2 = np.zeros((hidden_size,output_size))
super_b1 = np.zeros((batch_size,hidden_size))
super_b2 = np.zeros((batch_size,output_size))

nn = NeuralNetwork(epoch_size,batch_size,input_size,hidden_size,output_size,learn_rate)
error_avg = []
acc_count = 0

for e in range(epoch_size):
    count1 = 0
    for i in range(0,len(train_data_data),batch_size):
        batch = train_data_data[i:i+batch_size]
        
        label = train_label_data[i:i+batch_size]
        
        number_label = train_label[i:i+batch_size]
        
        loss,y,super_w1,super_w2,super_b1,super_b2 = nn.back_propagation(batch,label,number_label)
        
        for k in range(batch_size):
            if y[k][number_label[k]]>0.5:
                count1 = count1 + 1
        acc_count = acc_count + 1
        error_avg.append(loss)
#         if i % 10000 == 0:
#             print("Loss:",np.mean(error_avg))

    train_accuracy = (count1 * 100)/60000
    print("Train accuracy: {:.1f}%".format(train_accuracy))
    print("count:",count1)
    print('>Epoch=%d, lrate=%.6f, error=%.10f'%(e, learn_rate, np.mean(error_avg)))
print("train_error:",np.mean(error_avg))


train data shape: (60000, 784)
Train accuracy: 13.1%
count: 7874
>Epoch=0, lrate=0.000001, error=2.6265733664
Train accuracy: 38.7%
count: 23214
>Epoch=1, lrate=0.000001, error=2.0504948393
Train accuracy: 59.1%
count: 35476
>Epoch=2, lrate=0.000001, error=1.7307146846
Train accuracy: 69.3%
count: 41593
>Epoch=3, lrate=0.000001, error=1.5496477995
Train accuracy: 73.7%
count: 44218
>Epoch=4, lrate=0.000001, error=1.4654772847
train_error: 1.4654772847320847


In [10]:
count = 0
for j in range(0,len(test_data_data),batch_size):
    
    batch = test_data_data[j:j+batch_size]
    
    label = test_label[j:j+batch_size]
    
    h = np.dot(batch,super_w1) + super_b1
    
    H = nn.relu(h)
    
    z = np.dot(H,super_w2) + super_b2
    
    y = nn.softmax(z)
    
    if j % 1000 == 0:
        print("count:",count)
    
    for i in range(len(y)):
        if y[i][label[i]] > 0.5:
            count = count + 1

accuracy = (count * 100)/10000

print("Prediction accuracy: {:.1f}%".format(accuracy))

count: 0
count: 707
count: 1409
count: 2118
count: 2840
count: 3543
count: 4370
count: 5186
count: 5960
count: 6793
Prediction accuracy: 75.6%
